<a href="https://colab.research.google.com/github/EC523-Group-Project/sarcasm-detector/blob/main/headlines_bert.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#  !pip install -q git+https://github.com/huggingface/transformers.git

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 6.5 MB 8.6 MB/s 
     |████████████████████████████████| 77 kB 8.1 MB/s 
     |████████████████████████████████| 596 kB 64.4 MB/s 
     |████████████████████████████████| 895 kB 71.9 MB/s 


#523 Project

In [ ]:
try:
    import transformers
except ImportError as e:
    print('transformers not installed')
    print('Installing now...')
    !pip install -q git+https://github.com/huggingface/transformers.git
    pass 

In [ ]:
import torch
import io 
import os
from torch.utils.data import Dataset,DataLoader
from sklearn.metrics import classification_report,accuracy_score
import json
from tqdm.notebook import tqdm
from transformers.utils.dummy_pt_objects import AutoModelForSequenceClassification
from transformers import AutoModelForTokenClassification,AutoConfig, AutoModel,AutoTokenizer,BertModel,BertConfig,AdamW, get_constant_schedule,BertForSequenceClassification,get_linear_schedule_with_warmup
import random
import numpy as np

#Mounting Drive
from google.colab import drive
drive.mount('/content/gdrive')

#Set device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
#Data Directory 
DATA_DIR = '/content/gdrive/Shareddrives/523 Project/Data'

Mounted at /content/gdrive


In [ ]:
%cd '/content/gdrive/Shareddrives/523 Project/Data'
%ls

/content/gdrive/Shareddrives/523 Project/Data
Sarcasm_Headlines_Dataset.json     test-unbalanced.csv
Sarcasm_Headlines_Dataset_v2.json  train-balanced-sarcasm.csv
test-balanced.csv


In [ ]:

with open('Sarcasm_Headlines_Dataset_v2.json') as f:
    headlines = json.loads("[" + 
        f.read().replace("}\n{", "},\n{") + 
    "]")




In [ ]:

label = []
hd = []

for i in range(len(headlines)):
  label.append(headlines[i]['is_sarcastic'])
  hd.append(headlines[i]['headline'])




In [ ]:
#Class to store the headline articles

class Headlines(Dataset):

  def __init__(self,filepath,selected_tokenizer,max_length=None):

    headlines = []
    labels = []

    #potentially assign validation and test splits within this class

    
    # Funky JSON file, need to make slight adjustments reading it in 
    with open(str(filepath)) as f:
      data = json.loads("[" + 
          f.read().replace("}\n{", "},\n{") + 
      "]")

    #Store text and label information   
    for i in range(len(data)):
      labels.append(data[i]['is_sarcastic'])
      headlines.append(data[i]['headline'])

    self.length = len(headlines)

    if max_length == None:
      max_length = selected_tokenizer.model_max_length
    else:
      max_length = max_length

    #Create the tokenized version of our headlines

    self.inputs = selected_tokenizer(headlines,padding = True,truncation = True, add_special_tokens = True,return_tensors='pt')
    label_tensors = {'labels':torch.tensor(labels)}
    self.inputs.update(label_tensors)

    return

  #Required for the token's being passed to the transformer 
  def __getitem__(self,item):
    return {key: self.inputs[key][item] for key in self.inputs.keys()}
  def __len__(self):
    return self.length

In [ ]:
print(headlines_data[1]['input_ids'])

In [ ]:

#Defining the pretrained model (BERT)
config = AutoConfig.from_pretrained("bert-base-uncased")
selected_tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
bert_model = BertForSequenceClassification(config)

#cuda
bert_model.to(device)


In [ ]:
#Training data (need to split the data into train test validation, either can do this manually 
#in seperate folders, or assign splits randomly shuffled within the headlines class)
batch_size = 16


headlines_data = Headlines('Sarcasm_Headlines_Dataset_v2.json',selected_tokenizer = selected_tokenizer,max_length = 50)
trainloader = DataLoader(headlines_data, batch_size = batch_size, shuffle = True)


In [ ]:

#Optimizer and training
Epochs = 4
optimizer = AdamW(bert_model.parameters(),lr = 1e-3,eps = 1e-6)
scheduler = get_linear_schedule_with_warmup(optimizer,num_warmup_steps = 0,num_training_steps = len(trainloader)*Epochs)

predictions = []
labels = []

final_loss = 0

bert_model.train()

for epoch in range(1, Epochs+1):
  for batch in tqdm(trainloader,total = len(trainloader)):

    labels.extend(batch['labels'].numpy().flatten().tolist())
    # for key,value in batch.items():
    #   batch = {key:value.type(torch.long).to(device)}
    batch = {k:v.type(torch.long).to(device) for k,v in batch.items()}


    bert_model.zero_grad()

    outputs = bert_model(**batch)
    loss,logits = outputs[:2]

    final_loss += loss.item()

    loss.backward()
    optimizer.step()
    scheduler.step()

    logits = logits.detach().cpu().numpy()

    predictions.extend(logits.argmax(axis = -1).flatten().tolist())

  average_loss = final_loss/len(trainloader)


/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,


  0%|          | 0/1789 [00:00<?, ?it/s]

In [ ]:
# #Class to store the headline articles

# class Headlines(Dataset):

#   def __init__(self,filepath,selected_tokenizer,max_length=None):

#     headlines = []
#     labels = []



    
#     # Funky JSON file, need to make slight adjustments reading it in 
#     with open(str(filepath)) as f:
#       data = json.loads("[" + 
#           f.read().replace("}\n{", "},\n{") + 
#       "]")

#     random.shuffle(data)

    
#     #Store text and label information   
#     for i in range(len(data)):
#       label.append(data[i]['is_sarcastic'])
#       headlines.append(data[i]['headline'])

#     self.length = len(headlines)
#     if max_length == None:
#       max_length = selected_tokenizer.model_max_length
#     else:
#       max_length = max_length

#     #Create the tokenized version of our headlines

#     self.inputs = selected_tokenizer(headlines,padding = True,truncation = True, add_special_tokens = True,return_tensors='pt')
#     label_tensors = {'labels':torch.tensor(labels)}
#     self.inputs.update(label_tensors)

#     return

#     def __getitem__(self):
#       return {key: self.inputs[key][item] for key in self.inputs.keys()}
#     def __len__(self):
#       return self.length